# Seek NZ Scraper

###### Create a file with Jupyter and try and scrape Seek.co.nz
###### Used https://www.youtube.com/watch?v=eN_3d4JrL_w as a base
###### https://github.com/chrisgnorris/seeknz-scraper

## Import and setup

In [ ]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_url(position, location):
    """Generate a url from position and location"""
    template = 'https://www.seek.co.nz/{}-jobs/in-All-{}'
    url = template.format(position, location)
    return url


In [ ]:
url = get_url('data-scientist','New-Zealand')

## Extract raw html

In [ ]:
response = requests.get(url)

In [ ]:
response.reason

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
cards = soup.find_all('article')

In [ ]:
len(cards)

In [ ]:
section = soup.find('div', {'class':'_3MPUOLE'})
x = 0
for div in section.select('div[data-search-sol-meta]'):
    x = x + 1
print(x)

In [ ]:
jobs = soup.find_all('div[data-search-sol-meta]')

## Try to get Proxy via VPN working

In [ ]:
#Currently not working - obviously depends on the VPN provider and can be sticked. You can always just turn the VPN on
#Or you use the proxy - even some free ones https://pypi.org/project/free-proxy/
#This isn't required!!!

# https://dev.to/thughes24/how-to-turn-your-vpn-into-a-proxy-using-python-28ag
proxy = {
    'http': "test@gmail.com:password123@nz-akl.prod.surfshark.com",
    'https': "test@gmail.com:password123@nz-akl.prod.surfshark.com"
}
response2 = requests.get('https://google.com',proxies=proxy)

In [ ]:
response2.reason

In [ ]:
# use shadow socks from Surfshark
# https://stackoverflow.com/questions/56934030/how-do-i-use-nordvpn-servers-as-python-requests-proxies/65441742#65441742

## Prototype the model with a single record

In [ ]:
card = cards[0]

In [ ]:
atag = card.h1.a

In [ ]:
job_title = atag.string

In [ ]:
job_url = 'https://www.seek.co.nz' + atag.get('href')

In [ ]:
company = card.find('span',{"_3FrNV7v _3PZrylH E6m4BZb"}).a.text.strip()

In [ ]:
card.find('span',{"_3FrNV7v _3PZrylH E6m4BZb"}).text.strip()

In [ ]:
location = card.find('div',{'class':'xxz8a1h'}).a.text

In [ ]:
salary = card.find('span',{'class':'lwHBT6d'}).text

In [ ]:
card.find_all('span',{'class':{'Eadjc1o' : 'location'}})

In [ ]:
jobcategory = card.find(attrs={"data-automation": "jobClassification"}).text

## Prototype the model with a single record V2

In [ ]:
card = cards[1]

In [ ]:
job_title = card.find(attrs={"data-automation": "jobTitle"}).text

In [ ]:
job_url = 'https://www.seek.co.nz' + card.find(attrs={"data-automation": "jobTitle"}).get('href')

In [ ]:
try:
    company = card.find(attrs={"data-automation": "jobCompany"}).text
except AttributeError:
    company = ''
    

In [ ]:
location = card.find(attrs={"data-automation": "jobLocation"}).text

In [ ]:
try:
    job_salary = card.find(attrs={"data-automation": "jobSalary"}).text
except AttributeError:
    job_salary = ''

In [ ]:
job_category = card.find(attrs={"data-automation": "jobClassification"}).text

In [ ]:
job_subcategory = card.find(attrs={"data-automation": "jobSubClassification"}).text

In [ ]:
job_short_description = card.find(attrs={"data-automation": "jobShortDescription"}).text

In [ ]:
try:
    job_listing_date = card.find(attrs={"data-automation": "jobListingDate"}).text
except AttributeError:
    job_listing_date = 'Featured'

In [ ]:
date_mined = datetime.today().strftime('%Y-%m-%d')

In [ ]:
job_id = card.get('data-job-id')

In [ ]:
bullet_points = ''
x = 0
for li in card.select('li'):
    if x == 0:
        bullet_points = bullet_points + li.text
        x = 1
    else:
        bullet_points = bullet_points + ' - ' + li.text

## Generalise the model with a function

In [ ]:
def get_record(card):
    """Extract job data from a single record"""
    card = cards[0]
    job_title = card.find(attrs={"data-automation": "jobTitle"}).text
    job_url = 'https://www.seek.co.nz' + card.find(attrs={"data-automation": "jobTitle"}).get('href')
    company = card.find(attrs={"data-automation": "jobCompany"}).text
    location = card.find(attrs={"data-automation": "jobLocation"}).text
    try:
        job_salary = card.find(attrs={"data-automation": "jobSalary"}).text
    except AttributeError:
        job_salary = ''
    job_category = card.find(attrs={"data-automation": "jobClassification"}).text
    job_subcategory = card.find(attrs={"data-automation": "jobSubClassification"}).text
    job_short_description = card.find(attrs={"data-automation": "jobShortDescription"}).text
    try:
        job_listing_date = card.find(attrs={"data-automation": "jobListingDate"}).text
    except AttributeError:
        job_listing_date = 'Featured'
    date_mined = datetime.today().strftime('%Y-%m-%d')
    job_id = card.get('data-job-id')
    bullet_points = ''
    x = 0
    for li in card.select('li'):
        if x == 0:
            bullet_points = bullet_points + li.text
            x = 1
        else:
            bullet_points = bullet_points + ' - ' + li.text
            
    record = (job_id,job_title,company,location,jobshortdescription,bullet_points,job_salary,jobcategory,jobsubcategory,job_url)
    
    return record

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [ ]:
print(records[0])

## Getting the next page

In [ ]:
while True:
    try:
        url = 'https://www.seek.co.nz' + soup.find(attrs={"data-automation": "page-next"}).get('href')
    except AttributeError:
        break
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('article')
    
    for card in cards:
    record = get_record(card)
    records.append(record)

In [ ]:
print(len(records))

## Saving File to Excel and referencing as a dataframe to ignore previously mined files

In [ ]:
#Opening existing file as DB

try:
    df = pd.read_csv('results.csv')  
except FileNotFoundError:
    print('No existing file found')


In [ ]:
df

In [ ]:
#https://stackoverflow.com/questions/60675117/returning-a-string-from-loc-query
#https://stackoverflow.com/questions/56260348/selecting-single-value-in-a-pandas-dataframe
lookup = df.loc[df['ID'] == '1111', 'ID'].values

In [ ]:
lookup

In [ ]:
if lookup.size > 0:
    print('Found')
else:
    print('Not found')

In [ ]:
#Put together
def lookup_ID(jobid):
    """Feed URL for it to look up, will return found or not found"""
    
    try:
        df = pd.read_csv('results.csv')  
        lookup = df.loc[df['ID'] == jobid, 'ID'].values
        
        if lookup.size > 0:
            searchResult = 'Found'
         
        else:
            searchResult = 'Not found'
        
        return searchResult
        
    except FileNotFoundError:
        searchResult = 'No existing csv file found'
        return searchResult
        



In [ ]:
lookup_ID(51160410)

## Mine hidden salary data from Seek

In [ ]:
#https://stackoverflow.com/questions/37465172/how-to-use-beautifulsoup-to-scrape-a-webpage-url
import requests
from bs4 import BeautifulSoup

def get_url_salary(jobid_salary):
    """Generate a url to get salary info from seek job id"""
    template = 'https://qdjrmx4vb1.execute-api.ap-southeast-2.amazonaws.com/chickennuggets?jobId={}'
    url = template.format(jobid_salary)
    return url

get_url_salary(51155454)

In [ ]:
salurl = get_url_salary(51155454) # get url

response = requests.get(salurl) # get request
r = response.json() # convert the json array text to dictionary

print(r)

In [ ]:
type(r)

In [ ]:
print(r["jobInfoArray"])
print(r['jobInfoArray'][0]) #https://stackoverflow.com/questions/51788550/parsing-json-nested-dictionary-using-python

In [ ]:
r["jobInfoArray"][0]['upperLimit'] # dict name,item #, item name 

In [ ]:
salaryrange = str(r["jobInfoArray"][0]['upperLimit']) # dict name,item #, item name 

In [ ]:
salaryrange = salaryrange.replace(',', '') # Removes comma so can turn into int

In [ ]:
salaryrange = salaryrange.replace('-', ' ').split(' ') # splits text based on dash or space

In [ ]:
salaryrangelow = int(salaryrange[0])

In [ ]:
salaryrangehigh = int(salaryrange[1])

In [ ]:
print(salaryrangelow)
print(salaryrangehigh)

In [ ]:
# Combining it into a function

def get_salary(jobid_salary):
    """Generate a url to get salary info from seek job id and returns data as tuple with low and high"""
    
    template = 'https://qdjrmx4vb1.execute-api.ap-southeast-2.amazonaws.com/chickennuggets?jobId={}'
    salurl = template.format(jobid_salary)
    
    
    response = requests.get(salurl) # get request
    r = response.json() # convert the json array text to dictionary
    try:
        r["jobInfoArray"][0]['upperLimit'] # dict name,item #, item name 
        salaryrange = str(r["jobInfoArray"][0]['upperLimit']) # dict name,item #, item name
        salaryrange = salaryrange.replace(',', '') # Removes comma so can turn into int
        salaryrange = salaryrange.replace('-', ' ').split(' ') # splits text based on dash or space
        salaryrangelow = int(salaryrange[0])
        salaryrangehigh = int(salaryrange[1])
    
        return salaryrangelow,salaryrangehigh # return as a tuple - get away from global variables!
    
    except KeyError: #If it can't find job, will return 0
        salaryrangelow = int(0)
        salaryrangehigh = int(0)
    
        return salaryrangelow,salaryrangehigh # return as a tuple - get away from global variables!
    

In [ ]:
salaryresult = get_salary(51155454)

In [ ]:
print(salaryresult[0])
print(salaryresult[1])

## Putting it all together - Version 1

In [ ]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

#Global variables are telling Python that you are using these variables in the global pool. 
#Using them in local command can cause issues. Think of it as sending variables to the cloud, or local storage. I.e 
#sharepoint or saving on desktop
#https://www.w3schools.com/python/python_variables_global.asp 

def lookup_id(jobid):
    """Feed ID for it to look up, will return found or not found"""
    global df  #This is loaded in Main function, as not to load df everytime
    lookup = df.loc[df['ID'] == jobid, 'ID'].values
    
    if lookup.size > 0:
        searchResult = 'Found'
    else:
        searchResult = 'Not found'
    
    return searchResult

def get_salary(jobid_salary): # https://www.whatsthesalary.com/
    """Generate a url to get salary info from seek job id and returns data as tuple with low and high"""
    
    template = 'https://qdjrmx4vb1.execute-api.ap-southeast-2.amazonaws.com/chickennuggets?jobId={}'
    salurl = template.format(jobid_salary)
    
    
    response = requests.get(salurl) # get request
    r = response.json() # convert the json array text to dictionary
    try:
        r["jobInfoArray"][0]['upperLimit'] # dict name,item #, item name 
        salaryrange = str(r["jobInfoArray"][0]['upperLimit']) # dict name,item #, item name
        salaryrange = salaryrange.replace(',', '') # Removes comma so can turn into int
        salaryrange = salaryrange.replace('-', ' ').split(' ') # splits text based on dash or space
        salaryrangelow = int(salaryrange[0])
        salaryrangehigh = int(salaryrange[1])
    
        return salaryrangelow,salaryrangehigh # return as a tuple - get away from global variables!
    
    except (KeyError,NameError,TypeError): #If it can't find job, will return 0
        salaryrangelow = int(0) 
        salaryrangehigh = int(0)
    
        return salaryrangelow,salaryrangehigh # return as a tuple - get away from global variables!
    
def get_url(position, location):
    """Generate a url from position and location"""
    global search_word # this is for record function
    template = 'https://www.seek.co.nz/{}-jobs/in-All-{}'
    url = template.format(position, location)
    search_word = position
    return url


def get_record(card):
    """Extract job data from a single record"""
    global searchResult2 # this is in main function to instruct if CSV exists upon DF load attempt
    global search_word # this is from get_url function to add search word to record
    global result_to_print # this is ending some text to main to have a user friendly print statement
    searchResult = '' # needed only if there is no csv
    result_to_print = '' # needed only if there is no csv
    
    job_id = int(card.get('data-job-id')) # this is what df looks at - as it is unique id - get function as it is high level
    

    #If there IS a CSV file, run database lookup
    if searchResult2 != 'No existing csv file found': 
        searchResult = lookup_id(job_id)
        result_to_print = 'There is a CSV File - Record '+str(searchResult)
        
    #If there is no match found in df, OR NO CSV file, mine data
    if searchResult == 'Not found' or searchResult2 == 'No existing csv file found' : 
        
        job_title = card.find(attrs={"data-automation": "jobTitle"}).text
        job_url = 'https://www.seek.co.nz' + card.find(attrs={"data-automation": "jobTitle"}).get('href')
        
        try:
            company = card.find(attrs={"data-automation": "jobCompany"}).text
        except AttributeError:
            company = ''

        location = card.find(attrs={"data-automation": "jobLocation"}).text

        try:
            job_salary = card.find(attrs={"data-automation": "jobSalary"}).text
        except AttributeError:
            job_salary = ''

        job_category = card.find(attrs={"data-automation": "jobClassification"}).text
        job_subcategory = card.find(attrs={"data-automation": "jobSubClassification"}).text
        job_short_description = card.find(attrs={"data-automation": "jobShortDescription"}).text

        try:
            job_listing_date = card.find(attrs={"data-automation": "jobListingDate"}).text
        except AttributeError:
            job_listing_date = 'Featured'

        date_mined = datetime.today().strftime('%Y-%m-%d')
        
        salaryresult = get_salary(job_id) #Mine salary from https://www.whatsthesalary.com/
        salary_low = salaryresult[0]
        salary_high = salaryresult[1]
        
        bullet_points = ''
        x = 0
        for li in card.select('li'):
            if x == 0:
                bullet_points = bullet_points + li.text
                x = 1
            else:
                bullet_points = bullet_points + ' - ' + li.text
                
        #Adds all mined data to list
        record = (job_id,search_word,job_title,company,location,job_listing_date,date_mined,job_short_description,
                  bullet_points,job_salary,salary_low,salary_high,job_category,job_subcategory,job_url)
        
        # compiles all the text to let user know information on what happend
        result_to_print = result_to_print+' - Added as New Record'
       
       
        return record
    
    
def print_statusline(msg: str): #https://stackoverflow.com/a/43952192/6153315
    """Prints text inline and deletes properly"""    
    last_msg_length = len(print_statusline.last_msg) if hasattr(print_statusline, 'last_msg') else 0
    print(' ' * last_msg_length, end='\r')
    print(msg, end='\r')
#     sys.stdout.flush()  # Some say they needed this, I didn't.
    print_statusline.last_msg = msg
    

    
def main(position,location):
    """Run the main program routine"""
    global searchResult2 # sends this to record function to let it know if there isn't a CSV file
    global result_to_print # this is for user friendly print - from record function
    global df # this is sending the df to lookup_id function - so df is only loaded once
    searchResult2 = '' # else won't be defined if there is a CSV file causing errors
    
    # Loads CSV and dataframe
    try:
        df = pd.read_csv('results.csv')  
 
    except FileNotFoundError:
        searchResult2 = 'No existing csv file found'
        print(searchResult2)
     
    # Getting the url to mine    
    records = []
    url = get_url(position, location)
    print(url)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('article')
        x = 1
        for card in cards: 
            time.sleep(2)
            record = get_record(card)
            if record is not None:   
                records.append(record)   
            
            # this is printing some user friendly, inline text
            progressupdate = str(x)+'/'+str(len(cards)) +' - '+result_to_print
            result_to_print = ''
            print_statusline(progressupdate)

            x = x + 1
            
        # will try to get next seek page, until it can't find the next button    
        try:
            url = 'https://www.seek.co.nz' + soup.find(attrs={"data-automation": "page-next"}).get('href')
            print(url)
        except AttributeError:
            print('')
            print('no more pages, saving data')
            break
            
            
    # save the job data and creates titles/csv file if not already created
    if searchResult2 == 'No existing csv file found':
        with open('results.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['ID','SearchWord','JobTitle', 'Company', 'Location', 'DateListed', 'DateMined', 
                             'ShortDesc', 'BulletPoints','AdverSalary','LowSalary','HighSalary','Cat','SubCat','URL'])
            writer.writerows(records)
            print('saved and done')
    
    else:
        with open('results.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(records)
            print('saved and done')

In [ ]:
# run the main program
main('Python-Developer','New-Zealand')

#### Iterate through a list of searches

In [ ]:
# Ideally, should be in order of my granulality - as I want it label jobs as accurately as possible. Python will mine tons, while Python developer won't be as many.
# So start with specific keywords, then go broader. Exactly how you would search!

searchlist = ['Data-Scientist','Automation-Analyst','Continuous-Improvement-Analyst',
              'CI-Analyst','Operations-Analyst','Operation-Analyst', 'Python-Developer',
              'Commercial-Analyst','Business-Analyst','Data-Analyst','Python']


In [ ]:
for search in searchlist:
    print(search)
    main(search,'New-Zealand')